In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [7]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv("data/men_shoes.csv", low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [14]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])


(-58.133398968282776, 4.206122611474276)

In [15]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [16]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' :return output_dict

  features =  literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [26]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [28]:
def get_name_feat(key):
  return 'feat_' + key


for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df['features_parsed'].map(lambda feats: feats[key] if key in feats else np.nan ) 

In [29]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_black', 'feat_contained battery type', 'feat_rise',
       'feat_target audience', 'feat_nickel free', 'feat_light weight',
       'feat_gps', 'feat_power type', 'feat_number of pockets', 'feat_wash'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0]  * 100


In [41]:
{k: v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

  



In [68]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand].shape

(8846, 533)

In [0]:
feats = ['']

In [70]:
model = RandomForestRegressor(max_depth=5,n_estimators=100)
run_model(['brand_cat'], model)

(-57.30863703598273, 4.210915943427289)

In [85]:
feats_cat = [x for x in df.columns if 'cat' in x]

feats_cat

['categories',
 'brand_cat',
 'feat_location - country',
 'feat_clothing category',
 'feat_catalog',
 'feat_location - city/state',
 'feat_shoe category',
 'feat_multi pack indicator',
 'feat_certifications and listings',
 'feat_recommended location',
 'feat_fabrication',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_material_cat',
 'feat_manufacturer part number_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_waist_cat',
 'feat_product dimensions_cat',
 'feat_clothing type_cat',
 'feat_charge time (minutes)_cat',
 'feat_retail price_cat',
 'feat_����������������_cat',
 'feat_width_cat',
 'feat_is energy star-certified_cat',
 'feat_removable hood_cat',
 'feat_comfort technology_cat',
 'feat_is waterproof_cat',
 'feat_fuel type_cat',
 'feat_diameter_cat',
 'feat_adjustable_cat',
 'feat_accessory type_cat',
 'feat_suitable for_cat',
 'feat_machine washable_cat',
 'feat_materials_cat',
 'feat_safety features_cat',
 'feat_country of origin - components_cat',
 'feat_be

In [0]:
model = RandomForestRegressor(max_depth=5,n_estimators=100)

feats = ['brand_cat','feat_shape_cat','feat_metal type_cat','feat_brand_cat','feat_gender_cat','feat_material_cat', 'feat_style_cat']

result = run_model(feats, model)

In [98]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats)

(-57.276977463264494, 4.194846913507799)


Weight,Feature
0.2599 ± 0.0110,brand_cat
0.1017 ± 0.0100,feat_material_cat
0.0251 ± 0.0007,feat_gender_cat
0.0177 ± 0.0015,feat_brand_cat
0.0127 ± 0.0008,feat_shape_cat
0.0083 ± 0.0011,feat_metal type_cat
0.0030 ± 0.0011,feat_style_cat


In [81]:
df['brand'].value_counts(normalize=True)

nike             0.097210
puma             0.033315
ralph lauren     0.028775
vans             0.021116
new balance      0.020295
                   ...   
boconi           0.000055
jockey           0.000055
game time        0.000055
masimo capori    0.000055
smoky            0.000055
Name: brand, Length: 1732, dtype: float64

In [83]:
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'style': 'athletic sneakers', 'authenticity': '100% authentic guaranteed', 'material': 'synthetic', 'country/region of manufacture': 'china', 'mpn': '817416100', 'condition': 'new with box'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/sail/atomic orange/volt', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '620469018', 'brand': 'nike'},
       {'sport': 'basketball', 'style': 'basketball pants', 'condition': 'new with tags'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/volt', 'casual & dress shoe style': 'sandals', 'manufacturer part number': '343880033', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'condition': 'new with box', 'type': 'cleats'}], dtype=object)

In [99]:
!git add matrix_one/day5.ipynb

fatal: pathspec 'matrix_one/day5.ipynb' did not match any files
